In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_extract, length, greatest
from pyspark.sql.utils import AnalysisException

In [3]:
spark = (
    SparkSession
    .builder
    .master('local[2]')
    .config("spark.python.worker.reuse", "false")
    .config("spark.network.timeout", "600s")
    .config("spark.executor.heartbeatInterval", "300s")
    .getOrCreate()
)

## Exercise 2.2

In [3]:
exo2_2_df = spark.createDataFrame(
    [['test', 'more test', 10_000_000_000]], ['one', 'two', 'three']
)

In [ ]:
exo2_2_df.printSchema()

In [5]:
columns_to_check = exo2_2_df.columns

In [ ]:
type_counter = 0
for items in columns_to_check:
    if str(exo2_2_df.schema[items].dataType) != 'StringType()': 
        type_counter =+ 1

print(type_counter)

## Exercise 2.3

In [ ]:
exo2_3_df = (
    spark.read.text('../../data/gutenberg_books/1342-0.txt')
    .select(length(col('value')).alias('number_of_char'))
)

exo2_3_df.show()

## Exercise 2.4

In [3]:
exo2_4_df = spark.createDataFrame(
    [['key', 10_000, 20_000]], ['key', 'value1', 'value2']
)

In [ ]:
exo2_4_df.printSchema()

In [18]:
from pyspark.sql.utils import AnalysisException

try:
    exo2_4_mod = exo2_4_df.select(
        col('key'),
        greatest(col('value1'), col('value2')).alias('max_value')
    ).select('key', 'max_value')
except AnalysisException as err:
    print(err)

NameError: name 'exo2_4_df' is not defined

In [ ]:
exo2_4_mod.show(5)

## Exercise 2.5

In [10]:
book = spark.read.text('../../data/gutenberg_books/1342-0.txt')

lines = book.select(split(book.value, ' ').alias('line'))

words = lines.select(explode(col('line')).alias('word'))

words_lower = words.select(lower(col('word')).alias('word_lower'))

words_clean = words_lower.select(
    regexp_extract(col('word_lower'), '[a-z]*', 0).alias('word')
)

words_nonull = words_clean.where(col('word') != '')

In [12]:
words_not_is = words_nonull.where(col('word') != 'is')

In [13]:
words_longer_than_three = words_nonull.filter(
    length(col('word')) > 3
)

In [14]:
words_longer_than_three.show()

+------------+
|        word|
+------------+
|     project|
|   gutenberg|
|       ebook|
|       pride|
|   prejudice|
|        jane|
|      austen|
|        this|
|       ebook|
|      anyone|
|    anywhere|
|        cost|
|        with|
|      almost|
|restrictions|
|  whatsoever|
|        copy|
|        give|
|        away|
|       under|
+------------+
only showing top 20 rows



In [8]:
words_nonull.select(
    col('word'),
    length(col('word')).alias('word_length')
).show(10)

+----+-----------+
|word|word_length|
+----+-----------+
|   t|          1|
|   h|          1|
|   e|          1|
|   p|          1|
|   r|          1|
|   o|          1|
|   j|          1|
|   e|          1|
|   c|          1|
|   t|          1|
+----+-----------+
only showing top 10 rows



## Exercise 2.6

In [16]:
no_stop_words = words_nonull.where(~col('word').isin(['is', 'not', 'the', 'if']))
no_stop_words.show(5)

+---------+
|     word|
+---------+
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
+---------+
only showing top 5 rows



## Exercise 2.7

In [21]:
try:
    book = spark.read.text('../../data/gutenberg_books/1342-0.txt')
    # book = book.printSchema() remove this line for the code to work
    lines = book.select(split(book.value, ' ').alias('line'))
    words = lines.select(explode(col('line')).alias('word'))
except AnalysisException as err:
    print(err)